In [86]:
import pymolpro
import pandas as pd

In [87]:
backend = 'local' 
project_name = 'BHPERI_methodtest'
parallel = 4

In [89]:
methods = {"HF": "hf",
"MP2": "mp2",
"MP3": "mp3",
"MP4": "mp4",
"PAPT2":"PAPT;MP2",
"PAPT3":"PAPT;MP3",
"PAPT4":"PAPT;MP4",
"CCSD":"CCSD",
"CCSD(T)":"CCSD(T)" }
bases = ['cc-pVDZ','cc-pVTZ']

In [90]:
db = pymolpro.database.load("GMTKN55_BHPERI")

In [91]:
small=db.subset(open_shell=False,max_electrons=38)
print(small)

GMTKN55 BHPERI (closed shell only) (maximum number of electrons 38):

Molecules:
13ts_2a: {'geometry': '          10\n  \n    C          0.85850831      0.85149922     -0.05406558\n    C          0.85665794     -0.53234843     -0.02148388\n    N         -1.21895564     -0.93022557     -0.12349726\n    N         -1.66704575      0.25593414     -0.01212111\n    N         -1.21073866      1.33091420     -0.05063028\n    H          1.03972910      1.42014742      0.84839247\n    H          1.04072512      1.37132654     -0.98626823\n    H          1.02509073     -1.05697875      0.91129082\n    H          1.04207243     -1.10394180     -0.92050185\n    H         -1.76604359     -1.60632696      0.40888490\n', 'description': '13ts_2a'}
1,3-Cyclopentadiene: {'geometry': '          11\n  \n    H         -1.78935891      0.87878322      0.00009791\n    C         -1.12663167      0.00011091     -0.00000309\n    C         -0.19168134      0.00002491     -1.18134951\n    C          1.08468111    

In [92]:
results = {}
for method in methods:
    results[method] = {}
    for basis in bases:
        results[method][basis] = pymolpro.database.run(small, methods[method], basis, location=project_name,
                                                       backend=backend,
                                                       preamble="core,small", parallel=parallel)
        if results[method][basis].failed: print(method, basis, 'failed', results[method][basis].project_directory)
        
print(results)    

{'HF': {'cc-pVDZ': <pymolpro.database.Database object at 0x7f20e36fbee0>, 'cc-pVTZ': <pymolpro.database.Database object at 0x7f20e36f9510>}, 'MP2': {'cc-pVDZ': <pymolpro.database.Database object at 0x7f20e36fa3e0>, 'cc-pVTZ': <pymolpro.database.Database object at 0x7f2073354eb0>}, 'MP3': {'cc-pVDZ': <pymolpro.database.Database object at 0x7f20e36e9a50>, 'cc-pVTZ': <pymolpro.database.Database object at 0x7f20f85fff70>}, 'MP4': {'cc-pVDZ': <pymolpro.database.Database object at 0x7f20f88a0d60>, 'cc-pVTZ': <pymolpro.database.Database object at 0x7f20e37e12a0>}, 'PAPT2': {'cc-pVDZ': <pymolpro.database.Database object at 0x7f20e3f15ed0>, 'cc-pVTZ': <pymolpro.database.Database object at 0x7f20e3f17e20>}, 'PAPT3': {'cc-pVDZ': <pymolpro.database.Database object at 0x7f20e3f78250>, 'cc-pVTZ': <pymolpro.database.Database object at 0x7f20e3f78100>}, 'PAPT4': {'cc-pVDZ': <pymolpro.database.Database object at 0x7f20e3f79e40>, 'cc-pVTZ': <pymolpro.database.Database object at 0x7f20e3f7b220>}, 'CCSD':

In [93]:
analyse=pymolpro.database.analyse([result['cc-pVTZ'] for result in results.values()],small)
analyse['reaction statistics']

,HF/cc-pVTZ,MP2/cc-pVTZ,MP3/cc-pVTZ,MP4/cc-pVTZ,PAPT;MP2/cc-pVTZ,PAPT;MP3/cc-pVTZ,PAPT;MP4/cc-pVTZ,CCSD/cc-pVTZ,CCSD(T)/cc-pVTZ
MAD,0.030470,0.008147,0.006371,0.003814,0.006076,0.006317,0.000985,0.007783,0.000795
MAXD,0.040877,0.013025,0.012361,0.006694,0.011605,0.011834,0.003458,0.011974,0.002396
RMSD,0.031961,0.009027,0.007171,0.004250,0.006701,0.006918,0.001352,0.008172,0.001026
MSD,0.030470,-0.008147,0.006371,-0.003814,0.006076,0.006317,0.000418,0.007783,0.000727
STDEVD,0.010235,0.004123,0.003493,0.001989,0.002998,0.002991,0.001364,0.002643,0.000768


In [ ]:
import matplotlib.pyplot as plt

methods_pruned = [method for method in methods if (method != 'HF') and (method != 'CCSD(T)')]
bases_pruned = ['cc-pVTZ', 'cc-pVDZ']
fig, panes = plt.subplots(nrows=1, ncols=len(bases_pruned), sharey=True, figsize=(18, 6))

for pane in range(len(bases_pruned)):
    data = []
    for method in methods_pruned:
        data.append(
            pymolpro.database.analyse(results[method][bases_pruned[pane]],
                                      results['CCSD(T)']['cc-pVTZ'],'kJ/mol')['reaction energy deviations'].to_numpy()[:, 0]
        )
    panes[pane].violinplot(data, showmeans=True, showextrema=True, vert=True, bw_method='silverman')
    panes[pane].set_xticks(range(1, len(methods_pruned) + 1), labels=methods_pruned, rotation=-90)
    panes[pane].set_title(bases_pruned[pane])
panes[0].set_ylabel('Error / kJ/mol')
plt.savefig(project_name + ".violin.pdf")
df = pd.DataFrame(data)
df.to_excel("PAPT.xlsx")

In [95]:
extrapolate=pymolpro.database.basis_extrapolate(results[method].values(), results["HF"].values())
extrapolate